In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import duckdb
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import average_precision_score

In [2]:
%%time
train_path = '../train.parquet'
test_path = '../test.parquet'
df_test = pd.read_parquet("../test.parquet")

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 1min 51s
Wall time: 11.8 s


In [3]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,219336144,O=C(Nc1cnc(Cl)cc1C(=O)O)OCC1c2ccccc2-c2ccccc21,Cc1cc(CN)cc(Cl)n1.Cl.Cl,Cl.Cl.Cn1cncc1CN,Cc1cc(CNc2nc(NCc3cncn3C)nc(Nc3cnc(Cl)cc3C(=O)N...,BRD4,0
1,169574734,O=C(Nc1cc(Cl)ccc1C(=O)O)OCC1c2ccccc2-c2ccccc21,N#Cc1ccc(N)c([N+](=O)[O-])c1,COc1cc(C#N)c(F)cc1N,COc1cc(C#N)c(F)cc1Nc1nc(Nc2cc(Cl)ccc2C(=O)N[Dy...,HSA,0
2,267608955,O=C(O)C[C@H](Cc1ccc(Cl)c(Cl)c1)NC(=O)OCC1c2ccc...,Cn1ccnc1N,Nc1cccc(OCc2ccccc2)c1,Cn1ccnc1Nc1nc(Nc2cccc(OCc3ccccc3)c2)nc(N[C@H](...,BRD4,0
3,213966173,O=C(Nc1cccc(I)c1C(=O)O)OCC1c2ccccc2-c2ccccc21,Cc1cnc(N)cn1,Cc1cnc(F)c(CN)c1,Cc1cnc(F)c(CNc2nc(Nc3cnc(C)cn3)nc(Nc3cccc(I)c3...,sEH,0
4,117628504,O=C(N[C@H](CCC1CCCCC1)C(=O)O)OCC1c2ccccc2-c2cc...,Nc1ncc([N+](=O)[O-])s1,Cc1ccc(F)c(N)c1,Cc1ccc(F)c(Nc2nc(Nc3ncc([N+](=O)[O-])s3)nc(N[C...,HSA,0


In [4]:
%%time
# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

CPU times: total: 50.5 s
Wall time: 55.9 s


In [5]:
df['ecfp'].head()

0    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
1    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...
4    [0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: ecfp, dtype: object

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     60000 non-null  int64 
 1   buildingblock1_smiles  60000 non-null  object
 2   buildingblock2_smiles  60000 non-null  object
 3   buildingblock3_smiles  60000 non-null  object
 4   molecule_smiles        60000 non-null  object
 5   protein_name           60000 non-null  object
 6   binds                  60000 non-null  int64 
 7   molecule               60000 non-null  object
 8   ecfp                   60000 non-null  object
dtypes: int64(2), object(7)
memory usage: 4.1+ MB


In [7]:
RANDOM_STATE = 42

In [8]:
%%time
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

CPU times: total: 578 ms
Wall time: 647 ms


In [9]:
NUMBER_OF_MODELS = 3
# Create and train the ensemble models
catboost_model = CatBoostClassifier(iterations=500, random_state=RANDOM_STATE)
lgbm_model = LGBMClassifier(random_state=RANDOM_STATE)
xgb_model = XGBClassifier(random_state=RANDOM_STATE)

catboost_model.fit(X_train, y_train)
lgbm_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba_catboost = catboost_model.predict_proba(X_test)[:, 1]  
y_pred_proba_lgbm = lgbm_model.predict_proba(X_test)[:, 1]  
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1] 

# Average the predictions from the ensemble
y_pred_proba_ensemble = (y_pred_proba_catboost + y_pred_proba_lgbm + y_pred_proba_xgb) / NUMBER_OF_MODELS

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba_ensemble)
print(f"Mean Average Precision (mAP): {map_score:.8f}")

Learning rate set to 0.101594
0:	learn: 0.6571637	total: 216ms	remaining: 1m 47s
1:	learn: 0.6297494	total: 273ms	remaining: 1m 7s
2:	learn: 0.6090154	total: 337ms	remaining: 55.9s
3:	learn: 0.5882102	total: 385ms	remaining: 47.7s
4:	learn: 0.5720838	total: 455ms	remaining: 45s
5:	learn: 0.5575376	total: 496ms	remaining: 40.8s
6:	learn: 0.5471295	total: 538ms	remaining: 37.9s
7:	learn: 0.5368073	total: 623ms	remaining: 38.3s
8:	learn: 0.5282518	total: 694ms	remaining: 37.9s
9:	learn: 0.5215375	total: 750ms	remaining: 36.8s
10:	learn: 0.5139479	total: 817ms	remaining: 36.3s
11:	learn: 0.5086484	total: 870ms	remaining: 35.4s
12:	learn: 0.5022911	total: 928ms	remaining: 34.8s
13:	learn: 0.4982408	total: 994ms	remaining: 34.5s
14:	learn: 0.4935641	total: 1.04s	remaining: 33.6s
15:	learn: 0.4896988	total: 1.1s	remaining: 33.4s
16:	learn: 0.4837409	total: 1.16s	remaining: 33.1s
17:	learn: 0.4811154	total: 1.23s	remaining: 32.9s
18:	learn: 0.4781107	total: 1.29s	remaining: 32.7s
19:	learn: 0.

## Test

In [10]:
import os
import pandas as pd
from tqdm import tqdm
from rdkit import Chem

# Define the function to generate ECFPs
def generate_ecfp(mol):
    return Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)

output_file = 'submission.csv' 


# df_test = pd.read_parquet("../test.parquet")

In [11]:
# Generate ECFPs for the molecule_smiles
df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)


In [12]:
df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

In [13]:
# One-hot encode the protein_name
protein_onehot = pd.get_dummies(df_test['protein_name'])

In [14]:
batch_size = 1000  

# Iterate over the data in batches
X_test_batches = []
for i in range(0, len(df_test), batch_size):
    ecfp_batch = df_test['ecfp'].iloc[i:i+batch_size]
    protein_batch = protein_onehot.values[i:i+batch_size]
    X_test_batch = [list(ecfp) + list(protein) for ecfp, protein in zip(ecfp_batch, protein_batch)]
    X_test_batches.extend(X_test_batch)

X_test = X_test_batches

In [17]:
import numpy as np

# Define the batch size
batch_size = 1000

# Calculate the number of batches
num_batches = int(np.ceil(len(X_test) / batch_size))

# Initialize an empty array to store probabilities
probabilities_catboost = np.zeros(len(X_test))

# Predict on each batch and store the probabilities
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(X_test))
    batch_X_test = X_test[start_idx:end_idx]
    batch_probabilities = catboost_model.predict_proba(batch_X_test)[:, 1]
    probabilities_catboost[start_idx:end_idx] = batch_probabilities


In [18]:
# Define the batch size
batch_size = 1000

# Calculate the number of batches
num_batches = int(np.ceil(len(X_test) / batch_size))

# Initialize an empty array to store probabilities
probabilities_lgbm = np.zeros(len(X_test))

# Predict on each batch and store the probabilities
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(X_test))
    batch_X_test = X_test[start_idx:end_idx]
    batch_probabilities = lgbm_model.predict_proba(batch_X_test)[:, 1]
    probabilities_lgbm[start_idx:end_idx] = batch_probabilities


In [19]:
import numpy as np

# Define the batch size
batch_size = 1000

# Calculate the number of batches
num_batches = int(np.ceil(len(X_test) / batch_size))

# Initialize an empty array to store probabilities
probabilities_xgb = np.zeros(len(X_test))

# Predict on each batch and store the probabilities
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(X_test))
    batch_X_test = X_test[start_idx:end_idx]
    batch_probabilities = xgb_model.predict_proba(batch_X_test)[:, 1]
    probabilities_xgb[start_idx:end_idx] = batch_probabilities


In [20]:
# Average the predictions from the ensemble
probabilities_ensemble = (probabilities_catboost + probabilities_lgbm + probabilities_xgb) / NUMBER_OF_MODELS

In [28]:
probabilities_ensemble

array([0.207147  , 0.13942353, 0.11175329, ..., 0.16136476, 0.07326141,
       0.04787493])

In [21]:
# Output
output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities_ensemble})
output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))

In [31]:
# xgb_model ,lgbm_model, catboost_model save model to file
import joblib

joblib.dump(xgb_model, 'xgb_model.pkl')
lgbm_model.booster_.save_model('lgbm_model.txt')
catboost_model.save_model('catboost_model.bin')

In [36]:
df.to_csv("df_data.csv")
df_test.to_csv("df_test_data.csv")